# Workflow 1, Module 2

One approach to solving this module is to not define quite so tightly what's going on at the subquestion level in terms of enrichements and archetypes and so on, but simply pass the question to ROBOKOP and let its scoring bring the the best answers to the top.  Here we will use the quick service to start with a disease, find relevant phenotypes, and from there find genetic conditions. The answers will come out ranked by path.

For more details, see the "quick" notebook in greengamma/general.

First, we'll have a quick function that calls the quick service, and some functions for properly creating the question. 

In [3]:
#Load some functions for calling the quick service and parsing its output

import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from gg_functions import parse_answer, quick, get_view_url

#To make nicer looking outputs
from IPython.core.display import display, HTML

## Without pathway expansion

The module as written starts with a genetic condition, gets genes, and then gets chemicals interacting with those genes.   On the presumption that this is insufficient to get into new chemical space, it also adds the expansion from genes to other genes via pathways.   But lets originally start without the pathway expansion.  So we'll just do 

`genetic_condition` to `gene` to `chemical_substance`

In [3]:
def no_expand_question(disease_id):
    q = {
        "machine_question": {
            "nodes": [
                {
                    "id": "n0",
                    "type": "disease",
                    "curie": disease_id
                },
                {
                    "id": "n1",
                    "type": "gene"
                },
                {
                    "id": "n2",
                    "type": "chemical_substance"
                }
            ],
            "edges": [
                {
                    "id": "e0",
                    "source_id": "n0",
                    "target_id": "n1"
                },
                {
                    "id": "e1",
                    "source_id": "n1",
                    "target_id": "n2"
                }
            ]
        }
    }
    return q
    

In [5]:
glucose_intolerance = 'MONDO:0001076' #glucose intolerance
sids = 'MONDO:0010086'

In [5]:
glucose_intolerance_question = no_expand_question(glucose_intolerance)
gi_answer = quick(glucose_intolerance_question)

Return Status: 200


In [6]:
gi_frame = parse_answer(gi_answer,node_list=['n2'],edge_list=[])
gi_frame

,n2 - id,n2 - name,score
0,CHEBI:17234,glucose,0.804310
1,CHEBI:17234,glucose,0.785865
2,CHEBI:17234,glucose,0.759348
3,CHEBI:17234,glucose,0.759009
4,CHEBI:17234,glucose,0.720393
5,CHEBI:17234,glucose,0.719615
6,CHEBI:17234,glucose,0.663424
7,CHEBI:16469,17beta-estradiol,0.445326
8,CHEBI:9516,thapsigargin,0.440844
9,CHEBI:50122,rosiglitazone,0.438941


In [7]:
SIDS_question = no_expand_question(sids)
SIDS_answer = quick(SIDS_question)
SIDS_frame = parse_answer(SIDS_answer,node_list=['n2'],edge_list=[])
SIDS_frame

Return Status: 200


,n2 - id,n2 - name,score
0,CHEBI:28593,quinidine,0.441819
1,CHEBI:27958,cocaine,0.439788
2,CHEBI:28487,reserpine,0.438438
3,CHEBI:6809,methamphetamine,0.435212
4,CHEBI:33216,bisphenol A,0.431663
5,CHEBI:7983,pentobarbital,0.430733
6,CHEBI:27958,cocaine,0.429535
7,CHEBI:28790,serotonin,0.428917
8,CHEBI:6809,methamphetamine,0.427845
9,CHEBI:5118,fluoxetine,0.427508


## With pathway expansion

We can also run the question as asked by opening up the genes to those involved in the same biological processes

In [8]:
def pathway_expand_question(disease_id):
    q = {
        "machine_question": {
            "nodes": [
                {
                    "id": "n0",
                    "type": "disease",
                    "curie": disease_id
                },
                {
                    "id": "n1",
                    "type": "gene"
                },
                {
                    "id": "n2",
                    "type": "biological_process_or_activity"
                },
                {
                    "id": "n3",
                    "type": "gene"
                },
                {
                    "id": "n4",
                    "type": "chemical_substance"
                }
            ],
            "edges": [
                {
                    "id": "e0",
                    "source_id": "n0",
                    "target_id": "n1"
                },
                {
                    "id": "e1",
                    "source_id": "n1",
                    "target_id": "n2"
                },
                {
                    "id": "e2",
                    "source_id": "n2",
                    "target_id": "n3"
                },
                {
                    "id": "e3",
                    "source_id": "n3",
                    "target_id": "n4"
                }
            ]
        }
    }
    return q

In [9]:
gi_exp_question = pathway_expand_question(glucose_intolerance)
gi_exp_answer = quick(gi_exp_question,max_connectivity=500)
gi_exp_frame = parse_answer(gi_exp_answer,node_list=['n1','n2','n3','n4'],edge_list=[],node_properties=['name'])
view_url = get_view_url(gi_exp_answer)
display(HTML(f'<a href={view_url}>View Answer in ROBOKOP</a>'))
display(gi_exp_frame)

Return Status: 200


,n1 - name,n2 - name,n3 - name,n4 - name,score
0,LEPR,glucose homeostasis,INS,rosiglitazone,0.612847
1,LEP,glucose homeostasis,INS,rosiglitazone,0.612728
2,LEP,glucose homeostasis,INS,troglitazone,0.604530
3,SIRT1,regulation of cell proliferation,SIRT1,nicotinamide,0.592486
4,LEP,insulin secretion,LEP,troglitazone,0.588252
5,XBP1,endothelial cell proliferation,XBP1,thapsigargin,0.587243
6,PRKAA2,glucose homeostasis,PRKAA1,AICA ribonucleotide,0.587113
7,LEPR,negative regulation of gluconeogenesis,INS,rosiglitazone,0.587067
8,XBP1,fatty acid homeostasis,INS,rosiglitazone,0.586804
9,INSR,glucose homeostasis,INS,rosiglitazone,0.586480


In [12]:
sids_exp_question = pathway_expand_question(sids)
sids_exp_answer = quick(sids_exp_question,max_connectivity=500)
view_sids_url = get_view_url(sids_exp_answer)
display(HTML(f'<a href={view_sids_url}>View Answer in ROBOKOP</a>'))
sids_exp_frame = parse_answer(sids_exp_answer,node_list=['n0','n1','n2','n3','n4'],edge_list=[],node_properties=['name'])
display(sids_exp_frame)

Return Status: 200


,n0 - name,n1 - name,n2 - name,n3 - name,n4 - name,score
0,sudden infant death syndrome,TH,catecholamine biosynthetic process,TH,reserpine,0.557523
1,sudden infant death syndrome,TH,dopamine biosynthetic process,TH,reserpine,0.556578
2,sudden infant death syndrome,TH,tyrosine 3-monooxygenase activity,TH,reserpine,0.555430
3,sudden infant death syndrome,TH,response to hypoxia,TH,reserpine,0.555376
4,sudden infant death syndrome,TH,memory,TH,reserpine,0.555340
5,sudden infant death syndrome,TH,fatty acid metabolic process,TH,reserpine,0.554743
6,sudden infant death syndrome,TH,oxygen binding,TH,reserpine,0.554235
7,sudden infant death syndrome,TH,learning,TH,reserpine,0.554221
8,sudden infant death syndrome,TH,social behavior,TH,reserpine,0.554211
9,sudden infant death syndrome,TH,catecholamine biosynthetic process,TH,oxidopamine,0.546839


In [1]:
def pathway_expand_question_connected_process(disease_id):
    q = {
        "machine_question": {
            "nodes": [
                {
                    "id": "n0",
                    "type": "disease",
                    "curie": disease_id
                },
                {
                    "id": "n1",
                    "type": "gene"
                },
                {
                    "id": "n2",
                    "type": "biological_process_or_activity"
                },
                {
                    "id": "n3",
                    "type": "gene"
                },
                {
                    "id": "n4",
                    "type": "chemical_substance"
                }
            ],
            "edges": [
                {
                    "id": "e0",
                    "source_id": "n0",
                    "target_id": "n1"
                },
                {
                    "id": "e1",
                    "source_id": "n1",
                    "target_id": "n2"
                },
                {
                    "id": "e2",
                    "source_id": "n2",
                    "target_id": "n3"
                },
                {
                    "id": "e3",
                    "source_id": "n3",
                    "target_id": "n4"
                },
                {
                    "id": "e4",
                    "source_id": "n0",
                    "target_id": "n2"
                }
            ]
        }
    }
    return q

In [9]:
gi_exp_question = pathway_expand_question_connected_process(glucose_intolerance)
gi_exp_answer = quick(gi_exp_question,max_connectivity=1500)
gi_exp_frame = parse_answer(gi_exp_answer,node_list=['n1','n2','n3','n4'],edge_list=[],node_properties=['name'])
view_url = get_view_url(gi_exp_answer)
display(HTML(f'<a href={view_url}>View Answer in ROBOKOP</a>'))
display(gi_exp_frame)

Return Status: 200


,n1 - name,n2 - name,n3 - name,n4 - name,score
0,LEP,glucose metabolic process,INS,glucose,1.578499
1,INS,glucose metabolic process,INS,glucose,1.533790
2,INS,glucose metabolic process,LEP,glucose,1.517210
3,PRKAA1,glucose metabolic process,INS,glucose,1.473808
4,LEP,glucose metabolic process,LEP,glucose,1.471717
5,INS,glucose metabolic process,AKT1,glucose,1.461202
6,LEP,glucose metabolic process,AKT1,glucose,1.450712
7,INS,glucose metabolic process,PDX1,glucose,1.442872
8,PRKAA1,glucose metabolic process,LEP,glucose,1.423204
9,PRKAA1,glucose metabolic process,AKT1,glucose,1.410684


In [8]:
gi_exp_answer

<Response [500]>